In [1]:
#Imports
import json
import numpy
import sklearn 
#Base Naive Bayes classifier:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from evaluation import compute_macro_f1_score
from evaluation import compute_micro_f1_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import re
import pickle

In [2]:
import json
with open('data/train.json', 'r', encoding='utf-8') as fp:
        train_data = json.load(fp)
print(len(train_data ))

with open('data/valid.json', 'r', encoding='utf-8') as fp:
        valid_old = json.load(fp)

with open('data/valid_new.json', 'r', encoding='utf-8') as fp:
        valid_new = json.load(fp)
def get_text_langid(data):
        X = ['^' + i['text'] + '$' for i in data]
        y = [i['langid'] for i in data]
        return X, y
X_valid_old, y_valid_old = get_text_langid(valid_old)
X_valid_new, y_valid_new = get_text_langid(valid_new)
X_train, y_train = get_text_langid(train_data)
# X_valid_old_cnt = vec.transform(X_valid_old)
# X_valid_new_cnt = vec.transform(X_valid_new)

800003


## Setting up the predicted variable (language)

In [3]:
languages = set();
for i in train_data:
    languages.add(i['langid'])

languages = list(languages); #convert it into a list.
langind = {};
for i in languages:
    langind[i] = len(langind)
langname = {};
for i in langind:
    langname[langind[i]] = i;

### Applying a Simple NaiveBayes classifier at the start

In [5]:
def split_by_space(text): #A global function for consistent tokenization.
    return text[1:-1].lower().split(); #simple for now.

def split_by_punctuation(text):
    t = re.split(' |;|\*|\n|:|@|#|!|\(|\)', text[1:-1]);
    if(t[-1] == ''):
        t = t[:-1];
    return t;

#So we cannot do the bigram testing SINCE bigrams would be too big to handle. but we can maybe try tho

def unigram(text):
    return [x for x in text];
def quadgram(text):
    s = text.lower();
    return [text[i:i+4].lower() for i in range(len(text)-3)];
def pentagram(text):
    s = text.lower();
    return [s[i:i+5] for i in range(len(s)-4)];
def word_and_quadgram(text):
    s = text.lower();
    return split_by_space(s) + quadgram(s);
def word_and_pentagram(text):
    s = text.lower();
    return split_by_punctuation(s) + pentagram(s);
pentauniparameter = 1;
def penta_and_unigram(text):
    s = text.lower();
    return pentauniparameter*pentagram(s) + unigram(s);

def hexagram(text):
    s = text.lower();
    return [s[i:i+6] for i in range(len(s) - 5)];

def hexa_penta(text):
    return hexagram(text) + pentagram(text);


# def get_vocab(data):
#     vocab = set();
#     for i in range(len(data)):
#         strlist = unigram(data[i]['text']); #The string we need to determine the language of.
#         for j in strlist:
#             # if(j == 'त'):
#             #     print('found here: ', data[i]);
#             vocab.add(j); #we add all the words to the vocabulary.
#     return vocab
# vocab = get_vocab(train_data)

In [5]:
#we get each word in the vocabulary and assign it a unique index.
wordind = {};
for i in vocab:
    wordind[i] = len(wordind)

In [79]:
## Then we create a matrix of the size of the vocabulary and the number of languages.
## This denotes the frequencies of each word in the vocabulary for each of the languages.
def get_freq_matrix(data, wordind, langind):
    matrix = numpy.zeros((len(wordind), len(langind)));
    for i in range(len(data)):
        strlist = unigram(data[i]['text'])
        for j in strlist:
            matrix[wordind[j]][langind[data[i]['langid']]] += 1;
    return matrix
train_matrix = get_freq_matrix(train_data, wordind, langind)

In [6]:
def get_lang_cnts(data):
    langcnt = {};
    for i in data:
        if i['langid'] in langcnt:
            langcnt[i['langid']] += 1;
        else:
            langcnt[i['langid']] = 1;
    return langcnt
langcnt_train = get_lang_cnts(train_data)
langcnt_valid_new = get_lang_cnts(valid_new)
langcnt_valid_old = get_lang_cnts(valid_old)
def print_sorted_vals(d):
    total = 0;
    for i in d:
        total += d[i]
    print("total=", total)
    for i in sorted(d, key=d.get, reverse=True):
        print(i, "=", 100*d[i]/total, end = ",")
print("Training set language counts:")
print_sorted_vals(langcnt_train)
print("\nValidation set language counts:")
print("Old validation set:")
print_sorted_vals(langcnt_valid_old)
print("\nNew validation set:")
print_sorted_vals(langcnt_valid_new)

Training set language counts:
total= 800003
en = 39.47510196836762,fr = 10.297086385926052,de = 10.120087049673563,es = 9.813713198575506,it = 9.31634006372476,pt = 9.194090522160542,sv = 9.09934087747171,kn = 0.47349822438165856,bn = 0.4502483115688316,hi = 0.44599832750627183,ml = 0.44512333078750954,ta = 0.44462333266250254,mr = 0.424748407193473,
Validation set language counts:
Old validation set:
total= 100000
en = 39.475,fr = 10.297,de = 10.12,es = 9.814,it = 9.316,pt = 9.194,sv = 9.099,kn = 0.473,bn = 0.451,hi = 0.446,ta = 0.445,ml = 0.445,mr = 0.425,
New validation set:
total= 2446
en = 33.19705641864268,fr = 8.667211774325429,de = 8.503679476696648,es = 8.258381030253474,it = 7.849550286181521,pt = 7.726901062959935,sv = 7.686017988552739,mr = 4.170073589533933,bn = 3.0253475061324613,hi = 2.9026982829108747,ta = 2.7800490596892886,ml = 2.7800490596892886,kn = 2.4529844644317254,

In [4]:
vec_hexa_penta = CountVectorizer(tokenizer=hexa_penta)

In [5]:
vec_hexa_penta.fit(X_train);
X_train_vec = vec_hexa_penta.transform(X_train);
X_valid_old_vec = vec_hexa_penta.transform(X_valid_old);
X_valid_new_vec = vec_hexa_penta.transform(X_valid_new);

c:\Users\tripl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
del X_train, X_valid_old, X_valid_new

In [6]:
laplace_smoothing = 0.01;

In [7]:
def train(y_to_train = y_train, x_to_train = X_train_vec, nb = None):
    partial = True;
    if(nb == None):
        nb = MultinomialNB(alpha=laplace_smoothing)
        partial = False;
    if(partial):
        nb.partial_fit(x_to_train, y_to_train);
    else:
        nb.fit(x_to_train, y_to_train);
    return nb;

In [3]:
def run_tests(clf):
    y_pred_old_valid = clf.predict(X_valid_old_vec)
    y_pred_new_valid = clf.predict(X_valid_new_vec)
    y_pred_train = clf.predict(X_train_vec)
    print("accuracies: ", accuracy_score(y_valid_old, y_pred_old_valid), accuracy_score(y_valid_new, y_pred_new_valid), accuracy_score(y_train, y_pred_train))
    #print("confusion matrix: ", confusion_matrix(y_valid_old, y_pred_old_valid), confusion_matrix(y_valid_new, y_pred_new_valid), confusion_matrix(y_train, y_pred_train))
    print("micro F1  : ", compute_micro_f1_score(y_valid_old, y_pred_old_valid), compute_micro_f1_score(y_valid_new, y_pred_new_valid), compute_micro_f1_score(y_train, y_pred_train))
    print("macro F1  : ", compute_macro_f1_score(y_valid_old, y_pred_old_valid), compute_macro_f1_score(y_valid_new, y_pred_new_valid), compute_macro_f1_score(y_train, y_pred_train))
    print("-------------------------------------------------\n");
    return y_pred_train; #for soft computation.
    # return accuracy_score(y_valid_new, y_pred_new_valid);
    # print("Conf matrix for valid_new");
    # conf = confusion_matrix(y_valid_new, y_pred_new_valid, labels=languages);
    # disp = ConfusionMatrixDisplay(conf, display_labels=languages);
    # disp.plot();
    # plt.show();

In [ ]:
with open('savedir/vect.pickle', 'rb') as fp:
    vec = pickle.load(fp)

In [8]:
import pickle
X_train_vec = vec.transform(X_train)
X_valid_old_vec = vec.transform(X_valid_old)
X_valid_new_vec = vec.transform(X_valid_new)

In [ ]:
with open('savedir/model.pickle', 'rb') as fp:
    nb = pickle.load(fp)

In [9]:
run_tests(nb);

accuracies:  0.99484 0.9856909239574816 0.9944712707327348
micro F1  :  0.99150589319813 0.9787621359223301 0.9909015545320462
macro F1  :  0.9960655398541703 0.9880808975421688 0.9957983516745131
-------------------------------------------------



In [14]:
highest_acc = 0;
best_lap = 0.1;
for lap in range(1, 100, 2):
    laplace_smoothing = lap/100;
    clf = train();
    acc = run_tests(clf);
    if(acc > highest_acc):
        highest_acc = acc;
        best_clf = clf;
        best_lap = laplace_smoothing;
    print("new Best laplace smoothing: ", laplace_smoothing);    
    print("new Best accuracy: ", highest_acc);
print("Highest accuracy: ", highest_acc);
print("Best laplace smoothing: ", laplace_smoothing);

accuracies:  0.98663 0.9403107113654947 0.9945725203530487
micro F1  :  0.9780938180984058 0.9137625516834021 0.9910681592838071
macro F1  :  0.9769511078947956 0.9316449527740301 0.9943409648481386
-------------------------------------------------

new Best laplace smoothing:  0.02
new Best accuracy:  0.9403107113654947
accuracies:  0.98609 0.9370400654129191 0.9928087769670864
micro F1  :  0.977226588081205 0.909358446144791 0.9881912956732891
macro F1  :  0.9707740430347457 0.9272733044602783 0.9849475069566601
-------------------------------------------------

new Best laplace smoothing:  0.06
new Best accuracy:  0.9403107113654947
accuracies:  0.98577 0.9370400654129191 0.9918137806983224
micro F1  :  0.9767118355590468 0.9094117647058824 0.9865725439380644
macro F1  :  0.9685186546981891 0.9287010577293433 0.9800702287092946
-------------------------------------------------

new Best laplace smoothing:  0.1
new Best accuracy:  0.9403107113654947


KeyboardInterrupt: 

In [9]:
y_soft = [y_train]; x_soft = [X_train_vec];

In [10]:
nb = train(y_to_train=y_soft[-1], x_to_train = x_soft[-1]);

In [11]:
numpy.max(nb.feature_count_)

195589.0

In [12]:
for i in range(200):
    y_new = run_tests(nb); 
    y_soft.append(y_new);
    nb.feature_count_ = 1.1*nb.feature_count_;
    nb = train(y_to_train=y_soft[-1], x_to_train=x_soft[-1], nb=nb);
    print(i, "done");    


accuracies:  0.98577 0.9370400654129191 0.9918137806983224
micro F1  :  0.9767118355590468 0.9094117647058824 0.9865725439380644
macro F1  :  0.9685186546981891 0.9287010577293433 0.9800702287092947
-------------------------------------------------

0 done
accuracies:  0.98559 0.937857726901063 0.9912800326998774
micro F1  :  0.9764226578094833 0.9104301708898055 0.9857053281258324
macro F1  :  0.9643847981227918 0.9284620598655394 0.9772346989764169
-------------------------------------------------

1 done
accuracies:  0.98552 0.9407195421095667 0.9911537831733132
micro F1  :  0.9763085129010619 0.9143026004728133 0.9855004763514552
macro F1  :  0.964174724221135 0.9316419673368143 0.9764025379531168
-------------------------------------------------

2 done
accuracies:  0.9856 0.9415372035977105 0.9910975333842498
micro F1  :  0.9764355495917131 0.9154346540508574 0.9854093853585784
macro F1  :  0.965021841452408 0.9331096934721944 0.976034952926068
-----------------------------------

In [126]:
# import pickle
with open('sept_11_model.pkl', 'wb') as f:
    pickle.dump(nb, f)

In [127]:
nb.feature_count_.shape

(13, 6924248)

### OLD TESTS

## NEW TESTS